In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from dotenv import load_dotenv

load_dotenv('./env/.env')

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

# 메세지 토큰 수의 최대값
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=120, 
    memory_key="chat_history", 
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Michael")



WARNING! return_messages is not default parameter.
                    return_messages was transferred to model_kwargs.
                    Please confirm that return_messages is what you intended.




> Entering new LLMChain chain...


ValueError: variable chat_history should be a list of base messages, got 

In [42]:
chain.predict(question="I live in Korea")





> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Michael
AI: Hello Michael! How can I assist you today?
    Human:I live in Korea
    You:


> Finished chain.


"That's great to know! How can I assist you with information or tasks related to Korea?"

In [43]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Michael
AI: Hello Michael! How can I assist you today?
Human: I live in Korea
AI: That's great to know! How can I assist you with information or tasks related to Korea?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Michael.'